In [14]:
import hashlib
def sha512(message):
    hash_object = hashlib.sha512(message.encode())
    hex_dig = hash_object.hexdigest()
    return hex_dig

In [9]:
from Crypto.Cipher import AES 
from Crypto.Util.Padding import pad, unpad
import socket
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization

def format_key(key):
    return key.ljust(16)[:16].encode()

def encrypt(message, key):
    key = format_key(key)  
    iv = b'1234567890123456'
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return cipher.encrypt(pad(message.encode(), AES.block_size))

def decrypt(ciphertext, key):
    key = format_key(key)
    iv = b'1234567890123456'
    cipher = AES.new(key, AES.MODE_CBC, iv)
    return unpad(cipher.decrypt(ciphertext), AES.block_size).decode()

In [17]:
import socket
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization

def client():
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect(('localhost', 1234))

    client_private_key = ec.generate_private_key(ec.SECP256R1())
    client_public_key = client_private_key.public_key()

    server_public_bytes = client_socket.recv(1024)
    server_public_key = serialization.load_pem_public_key(server_public_bytes)

    client_public_bytes = client_public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    client_socket.sendall(client_public_bytes)
    
    shared_secret = client_private_key.exchange(ec.ECDH(), server_public_key)
    print("Client derived shared secret:", shared_secret.hex())
    
    msg = client_socket.recv(1024)
    decrypt_msg = decrypt(msg,shared_secret.hex())
    
    msg,hash_msg = decrypt_msg.split('***')
    hash2 = sha512(msg)
    
    if hash2 == hash_msg :
        print("Integrity is maintained!!")
        print(msg)
    else:
        print("Data is corrupted")
    
    
    client_socket.close()

client()

Client derived shared secret: 2ea18cea6d88fc29cf3ed9cc6fb63addbeaba31ce5fbb4c5cb7a5ea0f05ac0c1
Integrity is maintained!!
hi
